In [ ]:
# Computes the biomes and sub-biomes from rainfall, temperature and elevation on a PIXEL level

from PIL import Image
import cv2
import numpy as np
import pandas as pd
from numba import cuda
from geotiff import GeoTiff
from tensorflow import keras
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from tqdm.keras import TqdmCallback
from sklearn.metrics import accuracy_score

from inputs import models
from common import *

In [ ]:
MAPDIMS = (16384, 8192)
MIN_LAT = -60
MAX_LAT = 70

In [ ]:
waterMask = cv2.imread("outputs/waterMask.png", cv2.IMREAD_UNCHANGED)
Image.fromarray(waterMask).resize((400,200))

# Precipitation, temperature, elevation

### Average yearly rainfall in mm

In [ ]:
pre = np.array(GeoTiff('inputs/worldClim/pre1.tif').read(), dtype=np.float32)
for i in range(2,13):
    temp = np.array(GeoTiff(f'inputs/worldClim/pre{i}.tif').read(), dtype=np.float32)
    pre += temp
pre[pre<0] = np.NaN

pre = latitudeCropImage(pre, MIN_LAT, MAX_LAT)
pre = cv2.resize(pre, MAPDIMS, interpolation=cv2.INTER_LINEAR)

In [ ]:
maskTarget = np.isnan(pre)
maskValid = ~np.isnan(pre)
pre = GPUsearchAndFixToNearest(pre, maskTarget, maskValid).astype('float16')
print('Min:', np.min(pre), '; Max:', np.max(pre), '; Mean:', np.mean(pre), '[Rainfall per year in mm]')
smallNormImage(pre)

### Average yearly temperature

In [ ]:
tmp = np.array(GeoTiff('inputs/worldClim/tmp1.tif').read(), dtype=np.float32)
for i in range(2,13):
    temp = np.array(GeoTiff(f'inputs/worldClim/tmp{i}.tif').read(), dtype=np.float32)
    temp[temp<-1000] = -1000
    tmp += temp
tmp /= 12
tmp[tmp<-273] = np.NaN

tmp = latitudeCropImage(tmp, MIN_LAT, MAX_LAT)
tmp = cv2.resize(tmp, MAPDIMS, interpolation=cv2.INTER_LINEAR)

In [ ]:
maskTarget = np.isnan(tmp)
maskValid = ~np.isnan(tmp)
tmp = GPUsearchAndFixToNearest(tmp, maskTarget, maskValid).astype('float16')
print('Min:', np.min(tmp), '; Max:', np.max(tmp), '; Mean:', np.mean(tmp), '[Average annual temperature in Cº]')
smallNormImage(tmp)

### Elevation

In [ ]:
elv = np.array(GeoTiff('inputs/worldClim/elv2.tif').read(), dtype=np.float32)
elv[elv<-600] = np.NaN

elv = latitudeCropImage(elv, MIN_LAT, MAX_LAT)
elv = cv2.resize(elv, MAPDIMS, interpolation=cv2.INTER_LINEAR)

In [ ]:
maskTarget = np.isnan(elv)
maskValid = ~np.isnan(elv)
elv = GPUsearchAndFixToNearest(elv, maskTarget, maskValid).astype('float16')
print('Min:', np.min(elv), '; Max:', np.max(elv), '; Mean:', np.mean(elv), '[Height in m]')
smallNormImage(elv)

### Slope

In [ ]:
elvDiff = np.zeros(shape=(MAPDIMS[1],MAPDIMS[0]))
# Differentiating in X and Y (1st order, assuming distance = 1)
elvDiff[:,1:] += np.abs(elv[:,1:]-elv[:,0:-1])
elvDiff[1:,:] += np.abs(elv[1:,:]-elv[0:-1,:])
# Summing nearest cells
elvDiff[:,0:-1] += elvDiff[:,1:]
elvDiff[0:-1,:] += elvDiff[1:,:]

elvDiff[waterMask==255] = 0
smallNormImage(elvDiff)

In [ ]:
plt.hist(elvDiff.flatten(), bins=30, alpha=0.7)
plt.title('Histogram of slopes')
plt.xlabel('Slope magnitude')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
landMLinputs = pd.DataFrame(np.argwhere(waterMask == 0), columns=["y","x"])
landMLinputs["pre"] = pre[(waterMask == 0)]
landMLinputs["tmp"] = tmp[(waterMask == 0)]
landMLinputs.head(4)

# Biome classification

In [ ]:
biomeKeys = {
    'Tundra': (147,167,172),
    'Taiga': (91,143,82),
    'Cold desert': (146,126,48),
    'Shrubland': (179,124,6),
    'Temperate forest': (44,137,160),
    'Temperate rainforest': (10,84,109),
    'Desert': (200,113,55),
    'Savanna': (151,165,39),
    'Tropical Rainforest': (7,83,48),
    'Tropical forest': (0,127,14)
}

In [ ]:
nPoints = 600
bList = list(biomeKeys.keys())
X, Y2, Y = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
whittaker = np.array(Image.open('inputs/whittaker.png').convert('RGB'))
for i, biome in enumerate(biomeKeys):
    color = biomeKeys[biome]
    y, x = np.where((whittaker[:,:,0] == color[0]) & (whittaker[:,:,1] == color[1]) & (whittaker[:,:,2] == color[2]))
    index = np.random.choice(a=len(x), size=nPoints)

    X = pd.concat([X,pd.DataFrame({'Temperature': x[index]*20/399-10, 'Rain': -y[index]*2000/379+4649.077})], axis=0)
    Y2 = pd.concat([Y2,pd.DataFrame({'Biome': [biome]*nPoints})], axis=0)

X = X.reset_index(drop=True)
Y2 = Y2.reset_index(drop=True)
Y = pd.get_dummies(Y2.Biome)
Y = Y[bList]
pd.concat([X,Y], axis=1).head(3)

In [ ]:
#Cross-validation using the training samples to find the elbow of the curve (to avoid under or overfitting)
# params = {
#     'activations': ['leakyReLU','leakyReLU','softmax'],
#     'epochs': 400,
#     'loss': 'categorical_crossentropy',
#     'trainTestSplit': 0.05,
#     'batchSize': None,
#     'callbacks': None
# }

# ni = 1
# nf = 12
# yplotTrain = np.zeros(nf-ni+1)
# yplotTest = np.zeros(nf-ni+1)
# for i in tqdm(range(nf-ni+1)):
#     params['nodes'] = [2, i+ni, i+ni, 10]

#     model = models.autoencoder.kerasFeedForward(**params)
#     model.defineXnorm(minMax=True, minMaxData=X.to_numpy())

#     costs = model.fit(X, Y).history
#     yplotTrain[i], yplotTest[i] = costs['loss'][-1], costs['val_loss'][-1]

# plt.plot(range(ni,nf+1),yplotTrain,label='Training set error')
# plt.plot(range(ni,nf+1),yplotTest,label='Validation set error')
# plt.xlabel('Nº of neurons in hidden layer')
# plt.ylabel('Cost function error')
# plt.legend(loc='best')
# plt.show()

In [ ]:
params = {
    'nodes': [2, 8, 8, 10],
    'activations': ['leakyReLU','leakyReLU','softmax'],
    'epochs': 12000,
    'loss': 'categorical_crossentropy',
    'trainTestSplit': 0.05,
    'batchSize': None,
    'callbacks': [TqdmCallback()]
}
model = models.autoencoder.kerasFeedForward(**params)
model.defineXnorm(minMax=True, minMaxData=X.to_numpy())
model.fit(X, Y).history

model.plotLoss()
print(accuracy_score(Y, np.round(model.predict(X))))

In [ ]:
X = pd.DataFrame()
temp = [[],[]]
for y in range(whittaker.shape[0]):
    for x in range(whittaker.shape[1]):
        temp[0].append(x*20/399-10)
        temp[1].append(-y*2000/379+4649.077)
X['Temperature'] = temp[0]
X['Rainfall'] = temp[1]

Y = model.predict(X)

count = 0
for y in range(whittaker.shape[0]):
    for x in range(whittaker.shape[1]):
        biomeKey = np.argmax(Y[count])

        whittaker[y,x,:] = biomeKeys[bList[biomeKey]]
        count+=1

im = Image.fromarray(whittaker)
display(im)

### Biomes

In [ ]:
landMLinputs['biome'] = pd.DataFrame(model.predict(landMLinputs[['tmp','pre']].to_numpy())).idxmax(axis=1).astype('uint8')
display(landMLinputs.head())

biomes = len(biomeKeys)*np.ones(shape=(MAPDIMS[1],MAPDIMS[0]), dtype=np.uint8)
biomes[landMLinputs["y"], landMLinputs["x"]] = landMLinputs["biome"]

In [ ]:
biomeKeys = list(biomeKeys.values())
biomeKeys.append((1, 1, 154)) #10 ocean

biomesRGB = GPUpaintColorKeys(biomes, biomeKeys)

Image.fromarray(biomesRGB).save("outputs/pixelData/biomeDrawing.png")
smallNormImage(biomesRGB)

### Sub-biomes

In [ ]:
subBiomesKeys = biomeKeys.copy()

subBiomesKeys.append((0,0,0)) #11 rocky coast
subBiomesKeys.append((127,127,127)) #12 coast
subBiomesKeys.append((255,255,255)) #13 sandy coast

subBiomesKeys.append((219, 247, 255)) #14 tundra mountain
subBiomesKeys.append((163, 255, 147)) #15 taiga mountain
subBiomesKeys.append((25, 197, 255)) #16 temperate mountain
subBiomesKeys.append((255, 217, 84)) #17 dry mountain
subBiomesKeys.append((22, 255, 146)) #18 tropical mountain

In [ ]:
# materials = [
#     'beach sand',       #0
#     'rippled sand',     #1
#     'icelandic sand',   #2
#     'arid gravel',      #10
#     'arid soil',        #20
#     'mossy grass',      #30
# ]
# subMaterials = {
#     'Tundra': {2: 1},                   #0
#     'Taiga': {2: 1},                    #1
#     'Cold desert': {2: 1},              #2
#     'Shrubland': {10: 0.4, 30: 0.6},      #3
#     'Temperate forest': {2: 1},         #4
#     'Temperate rainforest': {2: 1},     #5
#     'Desert': {2: 1},                   #6
#     'Savanna': {2: 1},                  #7
#     'Tropical Rainforest': {2: 1},      #8
#     'Tropical forest': {2: 1},          #9
#     ### Coasts
#     'Black': {2: 1},                    #10
#     'Mixed': {0: 0.4, 2: 0.6},          #11
#     'White': {0: 0.5, 1: 0.5},          #12
#     ### Mountains
#     'Bare mountain': {2: 1},            #13
#     'Snowed mountain': {2: 1},          #14
#     'hilltest': {2: 1},
# }

# subMaterialsProbs = []
# for j,k in tqdm(enumerate(subMaterials)):
#     subMaterialsProbs.append([])
#     cumulative = 0
#     start = 0
#     for x in subMaterials[k]:
#         cumulative += subMaterials[k][x]
#         quantile = np.uint8(np.round(np.quantile(noiseSmall, q=cumulative)))-start
#         start += quantile
#         subMaterialsProbs[j].extend([x for _ in range(quantile)])
# subMaterialsProbs = np.array(subMaterialsProbs)

In [ ]:
def kernelAssignSubBiomes(yLimit: int, xLimit: int):
    @cuda.jit
    def func(biomes, elvDiff, waterMask):
        y, x = cuda.grid(2)
        
        # Out of bonds
        if y >= yLimit or x >= xLimit: return

        # Water
        if waterMask[y,x] == True: return

        cat = biomes[y,x]
        # Mountains
        if elvDiff[y,x] > 1000:
            if cat == 0: biomes[y,x] = 14
            elif cat == 1: biomes[y,x] = 15
            elif cat == 4 or cat == 5: biomes[y,x] = 16
            elif cat in [2,3,6,7]: biomes[y,x] = 17
            elif cat == 8 or cat == 9: biomes[y,x] = 18 

        # if cat==0 or cat==1 or cat==4: biomes[y,x] = 11
        # elif cat==6 or cat==8 or cat==9: biomes[y,x] = 13
        # else: biomes[y,x] = 12

    return func

def GPUassignSubBiomes(biomes: np.ndarray, elvDiff: np.ndarray, waterMask: np.ndarray):
    yLimit, xLimit = biomes.shape

    gpuBiomes = cuda.to_device(biomes)
    gpuElvDiff = cuda.to_device(elvDiff)
    gpuWaterMask = cuda.to_device(np.bool8(waterMask))

    threadsperblock = (16,16)
    blockspergrid = tuple(int(np.ceil(biomes.shape[i]/threadsperblock[i])) for i in range(2))

    kernelAssignSubBiomes(yLimit, xLimit)[blockspergrid, threadsperblock](gpuBiomes, gpuElvDiff, gpuWaterMask)
    subBiomes = gpuBiomes.copy_to_host()

    return subBiomes
subBiomes = GPUassignSubBiomes(biomes, elvDiff, waterMask)

In [ ]:
subBiomesRGB = GPUpaintColorKeys(subBiomes, subBiomesKeys)

Image.fromarray(subBiomesRGB).save("outputs/pixelData/subBiomeDrawing.png")
smallNormImage(subBiomesRGB)